# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [4]:
%pip install scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 131.4 kB/s eta 0:00:0000:0100:05
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import time
import datetime
import os
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file

IEX_CLOUD_API_TOKEN = os.getenv('IEX_KEY')
FZ_API_KEY = os.getenv('FZ_KEY')


## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [37]:
tickers = ['AAPL', 'TSLA', 'GOOG']


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [59]:
api_url = f"https://api.finazon.io/latest/finazon/us_stocks_essential/time_series"

now = datetime.datetime.now()
end_date = now
start_date = now - datetime.timedelta(days=365)
start_at = str(int(time.mktime(start_date.timetuple())))
end_at = str(int(time.mktime(end_date.timetuple())))

prices = {}

for t in tickers:
    querystring = {
        'ticker':t,
        'interval': '1d',
        'start_at': start_at,
        'end_at': end_at,
        'page_size': 500 # 251 # for two years minus non trading days (weekends and holidays)
    }

    api_key = f"apikey {FZ_API_KEY}"

    data = requests.get(api_url, headers={'Authorization':api_key}, params=querystring).json()['data']

    data = [
        [datetime.datetime.utcfromtimestamp(int(x['t'])).strftime('%Y-%m-%d %H:%M:%S'), round((x['h']+x['l']) / 2, 2)] for x in data
    ]

    ytd = data[0][1] / data[-1][1]
    prices[t] = [data[0][1], ytd, data]

/var/folders/jr/mhy81gpn45955yh7jff8xr_c0000gr/T/ipykernel_11164/740605139.py:25: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  [datetime.datetime.utcfromtimestamp(int(x['t'])).strftime('%Y-%m-%d %H:%M:%S'), round((x['h']+x['l']) / 2, 2)] for x in data


## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [8]:
cols = ['ticker','price','ytd','#']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [43]:
df = pd.DataFrame(columns=cols)

for k,v in prices.items():
    df = pd.concat(
        [
            pd.DataFrame([[
                k,
                v[0],
                v[1],
                'n/a'
            ]], columns=cols), 
            df
        ],
        ignore_index=True
    )
df

/var/folders/jr/mhy81gpn45955yh7jff8xr_c0000gr/T/ipykernel_11164/4025759420.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(


,ticker,price,ytd,#
0,GOOG,169.53,1.282957,n/a
1,TSLA,223.89,0.956264,n/a
2,AAPL,226.31,1.264090,n/a


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [58]:
df.sort_values('ytd', ascending =False, inplace=True)
# inplace actually modifies the existing df
df = df[:50]
# df.reset_index(inplace=True)
df

,level_0,index,ticker,price,ytd,#
0,0,0,GOOG,169.53,1.282957,n/a
1,1,2,AAPL,226.31,1.264090,n/a
2,2,1,TSLA,223.89,0.956264,n/a


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [75]:
def chunk(arr, k):
    res = []
    for i in range(len(arr)//k):
        res.append(arr[k*i:k*(i+1)])
    res.append(arr[k*(len(arr)//k-1+1):])
    return res


res = {}

for t in tickers:
    data = prices[t][2]
    month_chunks = chunk(data, 21)

    res[t] = {}
    for i in range(12):
        res[t][f'{i+1}m'] = month_chunks[i][0][1]/month_chunks[i][-1][1]
        # res[t][f'{i+1}m'] = month_chunks[i][0][0]

res

{'AAPL': {'1m': 1.0078378980182587,
  '2m': 1.0760776068049318,
  '3m': 1.1028183716075157,
  '4m': 1.148852657004831,
  '5m': 0.9487135209808324,
  '6m': 0.9730862563423781,
  '7m': 0.9362892365067854,
  '8m': 0.9692865903515944,
  '9m': 1.0318393727484638,
  '10m': 1.0766996925179364,
  '11m': 1.0072334806820138,
  '12m': 0.9867061386359828},
 'TSLA': {'1m': 0.893095057640911,
  '2m': 1.3628784554629223,
  '3m': 1.009126610985121,
  '4m': 1.2402203078661205,
  '5m': 0.8490770901194354,
  '6m': 0.886759670908069,
  '7m': 0.9138841727975854,
  '8m': 0.8228575898619315,
  '9m': 1.0998016899465426,
  '10m': 1.0546738790290133,
  '11m': 0.9260253542132738,
  '12m': 1.1297996839362747},
 'GOOG': {'1m': 0.9199587584111135,
  '2m': 1.0155860003327972,
  '3m': 0.9957389549226283,
  '4m': 1.1345077588399897,
  '5m': 1.0420094989631414,
  '6m': 1.0403417845636642,
  '7m': 0.955998116127296,
  '8m': 1.063791228706053,
  '9m': 0.9950948092832563,
  '10m': 0.9833858493268405,
  '11m': 1.0268072950

In [78]:
# Convert dictionary to DataFrame
df = pd.DataFrame(res).T.reset_index()
df.columns = ['ticker', '1m', '2m', '3m', '4m', '5m', '6m', '7m', '8m', '9m', '10m', '11m', '12m']

# Rank each column
ranked_df = df.set_index('ticker').rank(ascending=False)

# Cube low ranks and sum
ranked_df = ranked_df ** 3
ranked_df['cumulative_rank'] = ranked_df.sum(axis=1)

# Sort by cumulative rank and select the ones with the lowest numbers
final_df = ranked_df.sort_values('cumulative_rank')

# Display the final DataFrame
final_df

,1m,2m,3m,4m,5m,6m,7m,8m,9m,10m,11m,12m,cumulative_rank
ticker,,,,,,,,,,,,,
AAPL,1.0,8.0,1.0,8.0,8.0,8.0,8.0,8.0,8.0,1.0,8.0,27.0,94.0
GOOG,8.0,27.0,27.0,27.0,1.0,1.0,1.0,1.0,27.0,27.0,1.0,8.0,156.0
TSLA,27.0,1.0,8.0,1.0,27.0,27.0,27.0,27.0,1.0,8.0,27.0,1.0,182.0


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: